In [ ]:
from automatminer import MatPipe
from automatminer.automl.adaptors import SinglePipelineAdaptor, TPOTAdaptor
from automatminer.featurization import AutoFeaturizer
from automatminer.preprocessing import DataCleaner, FeatureReducer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from matminer.datasets.dataset_retrieval import load_dataset
import pandas as pd

In [ ]:
df_matminer_phonon = load_dataset('matbench_phonons')

In [ ]:
df_matminer_ph_dielec = load_dataset('phonon_dielectric_mp')
df_matminer_ph_dielec.set_index('mpid',inplace=True)

In [ ]:
#add mpid to structure objects in the phonon dataset
for row, col in df_matminer_ph_dielec.iterrows():
    for row2,col2 in df_matminer_phonon.iterrows():
        if col['structure'] == df_matminer_phonon.loc[row2,'structure']:
            df_matminer_phonon.loc[row2, 'mpid']= row

In [ ]:
df_matminer_phonon.set_index('mpid',inplace=True)
df = df_matminer_phonon

In [ ]:
#load lobster features data
lobpy_stat= pd.read_csv('Small_basis_sum_stat.csv', sep=';', index_col='mp-ids')
lobpy_stat.sort_index(inplace=True)

In [ ]:
# initialize matminer autofeaturizer to add features based on structure, composition and sine coloumb matrix
af =  AutoFeaturizer(n_jobs=10, preset="debug")
df = af.fit_transform(df,target ='last phdos peak')

In [ ]:
df

In [ ]:
# add lobster features to the dataframe
for feat in lobpy_stat.columns:
    for row, col in df.iterrows():
        df.loc[row,feat] = lobpy_stat.loc[row,feat]

In [ ]:
# Use DataCleaner from automatminer to obtain a clean dataset
dc = DataCleaner(feature_na_method="mean", max_na_frac=0.01, na_method_fit="drop", na_method_transform="mean")
df = dc.fit_transform(df, target ='last phdos peak')

In [ ]:
df.to_pickle('dataforml_automatminer.pkl')